In [1]:
import pandas as pd 
import geopandas as gpd 

In [3]:
data = pd.read_csv('./output/pred.csv')
columns = []
columns.append('uid')
for i in range(0,24):
    columns.append(i)
data.columns = columns

In [65]:
station_id = gpd.read_file('./data/上海充电桩_中促盟_合并站点.geojson')
station_id['充电站类型'].unique()

array(['01公共', '05其他专用', '06虚拟桩群', '02公交车专用', '04环卫物流车专用', '03出租车专用'],
      dtype=object)

In [8]:
station_id = gpd.read_file('./data/上海充电桩_中促盟_合并站点.geojson')
station_id.rename(columns={'经度':'lon','纬度':'lat','myid':'uid'},inplace=True)
station_id = station_id[['lon','lat','uid']]
station_id

,lon,lat,uid
0,120.893281,31.114590,8668
1,120.893938,31.110890,8561
2,120.913980,31.062432,6740
3,120.915917,31.030196,19011
4,120.916719,31.067247,6891
...,...,...,...
18332,121.937159,30.890878,18922
18333,121.939351,30.949531,4709
18334,121.942315,30.897054,149
18335,121.943069,30.915995,4360


In [10]:
data['uid'] = data['uid'].astype(int)

In [12]:
data = pd.merge(data,station_id,on='uid',how='inner')

In [13]:
data

,uid,0,1,2,3,4,5,6,7,8,...,16,17,18,19,20,21,22,23,lon,lat
0,1,1.093035,0.791152,0.605273,0.561252,0.494592,0.393992,0.375641,0.540498,0.828329,...,3.777033,3.195544,2.049654,1.134805,0.791782,0.950360,1.320145,1.472450,121.348137,30.715199
1,2,1.093032,0.791150,0.605270,0.561250,0.494592,0.393991,0.375641,0.540496,0.828327,...,3.777032,3.195543,2.049654,1.134804,0.791780,0.950357,1.320143,1.472448,121.344560,30.716703
2,3,1.093026,0.791145,0.605264,0.561244,0.494584,0.393985,0.375633,0.540488,0.828320,...,3.777021,3.195532,2.049642,1.134794,0.791772,0.950349,1.320134,1.472439,121.296063,30.717229
3,6,1.093035,0.791153,0.605274,0.561253,0.494593,0.393994,0.375644,0.540499,0.828332,...,3.777036,3.195547,2.049657,1.134808,0.791784,0.950361,1.320146,1.472451,121.344823,30.717439
4,7,1.093032,0.791150,0.605270,0.561250,0.494589,0.393991,0.375640,0.540496,0.828329,...,3.777033,3.195544,2.049653,1.134804,0.791781,0.950358,1.320143,1.472448,121.349360,30.719352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18304,19611,1.093049,0.791168,0.605288,0.561266,0.494607,0.394007,0.375657,0.540514,0.828346,...,3.777055,3.195566,2.049675,1.134825,0.791801,0.950377,1.320162,1.472466,121.508835,31.299307
18305,19612,1.093027,0.791145,0.605266,0.561246,0.494587,0.393987,0.375636,0.540492,0.828324,...,3.777028,3.195538,2.049650,1.134799,0.791776,0.950353,1.320138,1.472443,121.605119,31.313403
18306,19613,1.093037,0.791156,0.605277,0.561257,0.494597,0.393998,0.375649,0.540504,0.828336,...,3.777041,3.195551,2.049660,1.134810,0.791787,0.950364,1.320150,1.472456,121.520188,31.338381
18307,19615,1.093020,0.791139,0.605260,0.561238,0.494579,0.393977,0.375625,0.540482,0.828316,...,3.777019,3.195529,2.049639,1.134791,0.791770,0.950348,1.320134,1.472438,121.119527,31.362807


In [38]:
data.columns

Index([     'uid',          0,          1,          2,          3,          4,
                5,          6,          7,          8,          9,         10,
               11,         12,         13,         14,         15,         16,
               17,         18,         19,         20,         21,         22,
               23,      'lon',      'lat', 'geometry'],
      dtype='object')

#### 获取栅格级数据

In [21]:
import pandas as pd
import numpy as np
import geopandas as gpd
import transbigdata as tbd
import warnings
from shapely.geometry import Point
warnings.filterwarnings('ignore')

In [17]:
Chage_Grid = pd.read_csv('./data/订单量0712.csv')
Chage_Grid['总充电量'] = Chage_Grid['总充电量'].str.replace(',','').astype(float)
# 生成栅格范围
def generate_grid(grid_heatmap):
    grid_heatmap['lon'] = grid_heatmap['经纬度编号'].str.split('-').str[0].astype(float)
    grid_heatmap['lat'] = grid_heatmap['经纬度编号'].str.split('-').str[1].astype(float)
    params = {'slon': 120,
    'slat': 30,
    'deltalon': 0.01,
    'deltalat': 0.01,
    'theta': 0,
    'method': 'rect',
    'gridsize': 500}
    grid_heatmap['LONCOL'],grid_heatmap['LATCOL'] = tbd.GPS_to_grid(grid_heatmap['lon'], grid_heatmap['lat'],params)
    grid_heatmap['geometry'] = tbd.grid_to_polygon([grid_heatmap['LONCOL'], grid_heatmap['LATCOL']],params)
    grid_heatmap = gpd.GeoDataFrame(grid_heatmap, geometry='geometry')
    return grid_heatmap
Chage_Grid = generate_grid(Chage_Grid)

In [19]:
Chage_Grid['index'] = [i for i in range(len(Chage_Grid))]
Chage_Grid

,日期,充电时段,经纬度编号,充电次数,总充电量,BEV车辆数,PHEV车辆数,lon,lat,LONCOL,LATCOL,geometry,index
0,2023-07-12,0-1,121.46-31.34,100,3046.60,96,4,121.46,31.34,146,134,"POLYGON ((121.45500 31.33500, 121.46500 31.335...",0
1,2023-07-12,22-23,121.37-31.32,100,2144.80,58,40,121.37,31.32,137,132,"POLYGON ((121.36500 31.31500, 121.37500 31.315...",1
2,2023-07-12,22-23,121.57-31.16,98,2253.42,54,43,121.57,31.16,157,116,"POLYGON ((121.56500 31.15500, 121.57500 31.155...",2
3,2023-07-12,22-23,121.39-31.15,97,2603.53,67,30,121.39,31.15,139,115,"POLYGON ((121.38500 31.14500, 121.39500 31.145...",3
4,2023-07-12,22-23,121.4-31.29,85,1848.78,49,36,121.40,31.29,140,129,"POLYGON ((121.39500 31.28500, 121.40500 31.285...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4565,2023-07-12,8-9,121.36-31.32,1,8.50,0,1,121.36,31.32,136,132,"POLYGON ((121.35500 31.31500, 121.36500 31.315...",4565
4566,2023-07-12,8-9,121.54-31.14,1,9.14,1,0,121.54,31.14,154,114,"POLYGON ((121.53500 31.13500, 121.54500 31.135...",4566
4567,2023-07-12,8-9,121.57-31.15,1,13.50,0,1,121.57,31.15,157,115,"POLYGON ((121.56500 31.14500, 121.57500 31.145...",4567
4568,2023-07-12,8-9,121.47-31.14,1,12.92,0,1,121.47,31.14,147,114,"POLYGON ((121.46500 31.13500, 121.47500 31.135...",4568


In [20]:
charge_grid = gpd.GeoDataFrame(Chage_Grid,geometry='geometry')

In [24]:
data['geometry'] = data.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
data = gpd.GeoDataFrame(data,geometry='geometry')

In [25]:
sjoin = gpd.sjoin(charge_grid,data)

In [33]:
sjoin.columns

Index([         '日期',        '充电时段',       '经纬度编号',        '充电次数',
              '总充电量',      'BEV车辆数',     'PHEV车辆数',    'lon_left',
          'lat_left',      'LONCOL',      'LATCOL',    'geometry',
             'index', 'index_right',         'uid',             0,
                   1,             2,             3,             4,
                   5,             6,             7,             8,
                   9,            10,            11,            12,
                  13,            14,            15,            16,
                  17,            18,            19,            20,
                  21,            22,            23,   'lon_right',
         'lat_right'],
      dtype='object')

In [39]:
sjoin_use = sjoin[['LONCOL','LATCOL',0,          1,          2,          3,          4,
                5,          6,          7,          8,          9,         10,
               11,         12,         13,         14,         15,         16,
               17,         18,         19,         20,         21,         22,
               23]]

In [41]:
sjoin_use = sjoin_use.groupby(['LONCOL','LATCOL']).sum().reset_index()

In [42]:
sjoin_use

,LONCOL,LATCOL,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
0,134,123,10.930331,7.911512,6.052708,5.612505,4.945918,3.939922,3.756418,5.404973,...,22.886341,33.024115,37.770336,31.955445,20.496548,11.348054,7.917820,9.503590,13.201438,14.724484
1,134,124,3.279108,2.373460,1.815819,1.683757,1.483780,1.181980,1.126929,1.621498,...,6.865907,9.907238,11.331105,9.586637,6.148967,3.404420,2.375351,2.851082,3.960438,4.417355
2,135,118,17.488654,12.658540,9.684461,8.980136,7.913592,6.303990,6.010386,8.648079,...,36.618274,52.838718,60.432675,51.128846,32.794611,18.157019,12.668638,15.205864,21.122418,23.559307
3,135,119,68.861796,49.843210,38.132726,35.359421,31.159947,24.822216,23.666172,34.052128,...,144.184682,208.052697,237.953880,201.320036,129.128997,71.493457,49.882984,59.873343,83.169780,92.765005
4,135,120,19.674856,14.240970,10.895115,10.102741,8.902897,7.092105,6.761796,9.729201,...,41.195656,59.443654,67.986852,57.520031,36.894030,20.426743,14.252319,17.106696,23.762830,26.504321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,164,123,4.372130,3.164599,2.421078,2.244994,1.978350,1.575954,1.502552,2.161977,...,9.154525,13.209631,15.108117,12.782156,8.198598,4.539205,3.167117,3.801423,5.280567,5.889794
639,164,125,6.558191,4.746897,3.631618,3.367492,2.967538,2.363937,2.253829,3.242961,...,13.731771,19.814419,22.662144,19.173207,12.297871,6.808789,4.750664,5.702129,7.920848,8.834680
640,164,128,6.558214,4.746928,3.631648,3.367523,2.967569,2.363971,2.253863,3.242996,...,13.731826,19.814489,22.662226,19.173284,12.297948,6.808852,4.750709,5.702176,7.920883,8.834713
641,164,129,7.651193,5.538019,4.236860,3.928719,3.462103,2.757904,2.629449,3.783436,...,16.020391,23.116831,26.439183,22.368757,14.347532,7.943589,5.542429,6.652469,9.240964,10.307100


In [45]:
charge_grid['time'] = charge_grid['充电时段'].apply(lambda x: int(x.split('-')[0]))

In [48]:
charge_grid = charge_grid[['time','LONCOL','LATCOL','充电次数']]
charge_grid

,time,LONCOL,LATCOL,充电次数
0,0,146,134,100
1,22,137,132,100
2,22,157,116,98
3,22,139,115,97
4,22,140,129,85
...,...,...,...,...
4565,8,136,132,1
4566,8,154,114,1
4567,8,157,115,1
4568,8,147,114,1


In [59]:
tmp = charge_grid[charge_grid['time']==0]
tmp1 = sjoin_use[['LONCOL','LATCOL',0]]
tmp_merge = pd.merge(tmp,tmp1,on=['LONCOL','LATCOL'],how='inner')
# 归一化充电次数列
tmp_merge['real'] = (tmp_merge['充电次数'] - tmp_merge['充电次数'].min()) / (tmp_merge['充电次数'].max() - tmp_merge['充电次数'].min())

# 归一化0列
tmp_merge['pred'] = (tmp_merge[0] - tmp_merge[0].min()) / (tmp_merge[0].max() - tmp_merge[0].min())
def calculate_mape(row):
    actual = row["real"]
    forecast = row["pred"]
    if actual == 0:
        return 0
    else:
        return abs((actual - forecast) / actual) * 100
tmp_merge["mape"]=tmp_merge.apply(calculate_mape,axis=1)
average_mape=tmp_merge["mape"].mean()
average_mape


496.2472861848094

In [61]:
for t in charge_grid['time'].unique():
    tmp = charge_grid[charge_grid['time']==t]
    tmp1 = sjoin_use[['LONCOL','LATCOL',t]]
    tmp_merge = pd.merge(tmp,tmp1,on=['LONCOL','LATCOL'],how='inner')
    # 归一化充电次数列
    tmp_merge['real'] = (tmp_merge['充电次数'] - tmp_merge['充电次数'].min()) / (tmp_merge['充电次数'].max() - tmp_merge['充电次数'].min())

    # 归一化0列
    tmp_merge['pred'] = (tmp_merge[t] - tmp_merge[t].min()) / (tmp_merge[t].max() - tmp_merge[t].min())
    def calculate_mape(row):
        actual = row["real"]
        forecast = row["pred"]
        if actual == 0:
            return 0
        else:
            return abs((actual - forecast) / actual) * 100
    tmp_merge["mape"]=tmp_merge.apply(calculate_mape,axis=1)
    average_mape=tmp_merge["mape"].mean()
    print(f'time-{t} hour , mape: {average_mape}')

time-0 hour , mape: 496.2472861848094
time-22 hour , mape: 145.4630910904379
time-14 hour , mape: 368.8832768495303
time-19 hour , mape: 183.0890153749986
time-5 hour , mape: 462.8746157051849
time-11 hour , mape: 224.43473361776293
time-21 hour , mape: 157.05740141050165
time-8 hour , mape: 139.9739136213749
